# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name: Topaz Montague - [Project Repository Link](https://github.com/tfmontague/article-summarizer)

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

1. Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

###### Response to Question 1:

In [25]:
import requests
from bs4 import BeautifulSoup
import os

# Define the URL of the webpage
url = 'https://www.thinkful.com/blog/navigating-a-career-change-into-data-science-a-step-by-step-guide/'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the div element that contains the article
    article_div = soup.find('div', class_="text")
    
    if article_div:
        # Create the data directory if it does not exist
        if not os.path.exists('data'):
            os.makedirs('data')
        
        # Define the path to the output HTML file
        output_file_path = os.path.join('data', 'article.html')
        
        # Write the article HTML to the file
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(str(article_div))
        print(f"Article saved to {output_file_path}")
    else:
        print("Article content not found.")
else:
    print(f"Failed to retrieve webpage: Status code {response.status_code}")


Article saved to data\article.html


2. Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

###### Response to Question 2:

In [26]:
import os
from bs4 import BeautifulSoup
from textblob import TextBlob

# Load the article HTML from the file
file_path = os.path.join('data', 'article.html')
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML to extract text
soup = BeautifulSoup(html_content, 'html.parser')
article_text = soup.get_text()

# Perform sentiment analysis
blob = TextBlob(article_text)
sentiment = blob.sentiment.polarity  # Gives a measure of the sentiment

# Print the polarity score
print(f"Sentiment Polarity Score: {sentiment}")

# Count the number of sentences by splitting on sentence terminators
sentences = [sent.strip() for sent in article_text.replace('?', '.').replace('!', '.').split('.') if sent.strip()]
print(f"Number of sentences in the article: {len(sentences)}")


Sentiment Polarity Score: 0.151489898989899
Number of sentences in the article: 58


3. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

###### Response to Question 3:

In [27]:
import os
from bs4 import BeautifulSoup
import spacy
from collections import Counter
from prettytable import PrettyTable

# Load spaCy's English-language model
nlp = spacy.load('en_core_web_sm')

# Load the article HTML from the file
file_path = os.path.join('data', 'article.html')
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML to extract text
soup = BeautifulSoup(html_content, 'html.parser')
article_text = soup.get_text()

# Process the text with spaCy
doc = nlp(article_text)

# Tokenize and count words, filtering for nouns, adjectives, and verbs
words = [token.text.lower() for token in doc if token.is_alpha and token.pos_ in ['NOUN', 'ADJ', 'VERB']]
word_freq = Counter(words)

# Determine the 5 most frequent tokens
most_common_tokens = word_freq.most_common(5)

# Prepare and print the common tokens and their frequencies in a table with borders
table = PrettyTable()
table.field_names = ["Token", "Frequency"]
for token, freq in most_common_tokens:
    table.add_row([token, freq])

print("Common Tokens and Their Frequencies:")
print(table)

# Optionally, print all tokens and their frequencies if needed
full_table = PrettyTable()
full_table.field_names = ["Token", "Frequency"]
for token, freq in word_freq.items():
    full_table.add_row([token, freq])

print("\nAll Tokens and Their Frequencies:")
print(full_table)


Common Tokens and Their Frequencies:
+---------+-----------+
|  Token  | Frequency |
+---------+-----------+
|   data  |     27    |
| science |     25    |
|  career |     8     |
|  skills |     8     |
|  field  |     6     |
+---------+-----------+

All Tokens and Their Frequencies:
+--------------------+-----------+
|       Token        | Frequency |
+--------------------+-----------+
|      exciting      |     1     |
|       nerve        |     1     |
|      racking       |     1     |
|       embark       |     1     |
|       career       |     8     |
|       change       |     2     |
|     venturing      |     1     |
|      dynamic       |     1     |
|      evolving      |     4     |
|        data        |     27    |
|      science       |     25    |
|     landscape      |     3     |
|      combines      |     1     |
|     technology     |     2     |
|     statistics     |     2     |
|       domain       |     2     |
|     expertise      |     2     |
|      prese

4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?